![logo_ironhack_blue 7](https://user-images.githubusercontent.com/23629340/40541063-a07a0a8a-601a-11e8-91b5-2f13e4e6b441.png)

# Lab | Inferential statistics - T-test & P-value

## Instructions

1. *One tailed t-test* - In a packing plant, a machine packs cartons with jars. It is supposed that a new machine will pack faster on the average than the machine currently used. To test that hypothesis, the times it takes each machine to pack ten cartons are recorded. The results, in seconds, are shown in the tables in the file `files_for_lab/machine.txt`.
   Assume that there is sufficient evidence to conduct the t test, does the data provide sufficient evidence to show if one machine is better than the other?

2. *Matched Pairs Test* - In this challenge we will compare dependent samples of data describing our Pokemon (file `files_for_lab/pokemon.csv`). Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. Compare the two columns to see if there is a statistically significant difference between them and comment your result.


## Inferential statistics - ANOVA

Note: The following lab is divided in 2 sections which represent activities 3 and 4.

### Part 1

In this activity, we will look at another example. Your task is to understand the problem and write down all the steps to set up ANOVA. After the next lesson, we will ask you to solve this problem using Python. Here are the steps that you would need to work on:
    - Null hypothesis
    - Alternate hypothesis
    - Level of significance
    - Test statistic
    - P-value
    - F table

#### Context

Suppose you are working as an analyst in a microprocessor chip manufacturing plant. You have been given the task of analyzing a plasma etching process with respect to changing Power (in Watts) of the plasma beam. Data was collected and provided to you to conduct statistical analysis and check if changing the power of the plasma beam has any effect on the etching rate by the machine. You will conduct ANOVA and check if there is any difference in the mean etching rate for different levels of power. You can find the data `anova_lab_data.xlsx` file in the `files_for_lab` folder  

- State the null hypothesis
- State the alternate hypothesis
- What is the significance level
- What are the degrees of freedom of model, error terms, and total DoF

Data was collected randomly and provided to you in the table as shown: [link to the image - Data](https://education-team-2020.s3-eu-west-1.amazonaws.com/data-analytics/7.05/7.05-lab_data.png)


### Part 2

- In this section, use the Python to conduct ANOVA.
- What conclusions can you draw from the experiment and why?

## Imports

In [ ]:
import numpy as np
import pandas as pd

import scipy.stats as st

## One Tailed T-Test

Hypothesis
$$H0: \mu_{New} <= \mu_{Old}$$
$$H1: \mu_{New} > \mu_{Old}$$

In [ ]:
machine = pd.read_csv("files_for_lab/machine.txt", encoding = "utf-16", sep = "\t", names = ["New", "Old"], header=0)
machine

In [ ]:
machine["New"].mean(), machine["Old"].mean()

In [ ]:
ab_t, ab_p = st.ttest_ind(machine["New"], machine["Old"], alternative="greater", equal_var=False)
ab_t, ab_p

In [ ]:
ab_cv = st.t.ppf(0.95, df = len(machine["New"])+len(machine["Old"]-2))
ab_cv

In [ ]:
ab_t < ab_cv

In [ ]:
ab_a = st.t.cdf(ab_t, df = len(machine["New"])+len(machine["Old"]-2))
ab_a

In [ ]:
ab_a < 0.95

H0 is accepted, the new machine is faster

## Matched Pairs Test

Hypothesis:
$$H0: \mu_{Attack} = \mu_{Defense} → \mu_{Attack} - \mu_{Defense} = 0$$
$$H1: \mu_{Attack} \neq \mu_{Defense} → \mu_{Attack} - \mu_{Defense} \neq 0$$

In [ ]:
pokemon = pd.read_csv("files_for_lab/pokemon.csv", index_col = 0)
pokemon.head()

In [ ]:
pokemon["Attack"].mean(), pokemon["Defense"].mean()

In [ ]:
mp_t, mp_p = st.ttest_rel(pokemon["Attack"], pokemon["Defense"])
mp_t, mp_p

In [ ]:
mp_cv_u = st.t.ppf(1-(0.05/2), df = pokemon.shape[0]-1)
mp_cv_l = st.t.ppf((0.05/2), df = pokemon.shape[0]-1)
mp_cv_u, mp_cv_l

In [ ]:
mp_cv_u >= mp_t >= mp_cv_l

In [ ]:
mp_a = st.t.cdf(mp_t, df = pokemon.shape[0]-1)
mp_a

In [ ]:
(1-(0.05/2)) >= mp_a >= (0.05/2)

H0 is rejected, Attack and Defense are not equal

## ANOVA

### Part 1

- Null hypothesis: there is NO difference in etching rate
- Alternate hypothesis: there is difference in etching rate
- Level of significance: 0.05

In [ ]:
anova = pd.read_excel("files_for_lab/anova_lab_data.xlsx", header = 0, names = ["Power", "Rate"])
anova.head()

In [ ]:
anova["Power"] = pd.to_numeric(anova["Power"].apply(lambda x: x.strip(" W")))
anova.head()

In [ ]:
group_anova = anova.groupby("Power")["Rate"].agg(Rate="mean", Samples="size").reset_index()
group_anova

In [ ]:
# Degrees of Freedom
df = len(anova)-len(group_anova)
df

### Part 2

#### S2T and S2E

In [ ]:
S2T = 0
for power in anova['Power'].unique():
    ng = len(anova[anova['Power'] == power])  
    S2T  += ((anova[anova['Power'] == power]['Rate'].mean() - anova['Rate'].mean())**2)*ng
S2T /= (anova['Power'].nunique()-1)
print("The value of S2T is {:.2f}".format(S2t))

In [ ]:
S2E = 0
for power in anova['Power'].unique():
    for rate in anova[anova['Power'] == power]['Rate']:
        S2E += (rate - anova[anova['Power'] == power]['Rate'].mean())**2
S2E /= (len(anova) - anova['Power'].nunique()) 

print()
print("The value of S2E is {:.2f}".format(S2E))

#### F

In [ ]:
F = S2T / S2E
print("The value of F is {:.2f}".format(F))

In [ ]:
anova_t, anova_p = st.f_oneway(anova[anova['Power'] == 160]['Rate'],anova[anova['Power'] == 180]['Rate'],anova[anova['Power'] == 200]['Rate'])
anova_t, anova_p

#### Conclusion

In [ ]:
anova_p > 0.05

H0 is rejected, therefore there IS a difference in etching rate between different powers